Notebook version

In [1]:
import requests
from datetime import datetime, timedelta
from collections import defaultdict

# OpenWeather API setup
API_KEY = 'fd94c86864c1809c326f7f0b6add6acc'

def get_forecast(location, time):
    # Define the base URL with location as postal code and country code
    base_url = f"http://api.openweathermap.org/data/2.5/forecast?zip={location},GB&appid={API_KEY}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    if data['cod'] != '200':
        print(f"Failed to retrieve data for {location}. Please check the location and try again.")
        return None

    # Initialize variables for checking rain earlier in the day
    rain_earlier = False
    closest_entry = None
    min_time_diff = timedelta(hours=3)  # 3-hour interval sets the threshold for closest match

    for entry in data['list']:
        forecast_time = datetime.fromtimestamp(entry['dt'])
        time_diff = abs(forecast_time - time)

        # Check if there was rain earlier in the day
        if forecast_time < time and 'rain' in entry and entry['rain'].get('3h', 0) > 0:
            rain_earlier = True

        # Find the closest forecast entry to the specified time
        if time_diff < min_time_diff:
            min_time_diff = time_diff
            closest_entry = entry

    if closest_entry:
        forecast_time = datetime.fromtimestamp(closest_entry['dt'])
        rain_forecast = 'rain' in closest_entry and closest_entry['rain'].get('3h', 0) > 0
        return {
            "time": forecast_time,
            "wind_speed": closest_entry['wind']['speed'],
            "temperature": closest_entry['main']['temp'],
            "description": closest_entry['weather'][0]['description'],
            "rain_forecast": rain_forecast,
            "rain_earlier": rain_earlier
        }
    return None

def calculate_tennis_score(temp, wind_speed, rain, description):
    score = 100

    # Temperature scoring
    ideal_temp = 15
    temp_deviation = abs(temp - ideal_temp)
    score -= min(temp_deviation * 2, 20)  # Penalize deviations, up to 20 points

    # Wind scoring
    ideal_wind = 0
    wind_penalty = min(wind_speed * 10, 30)  # Higher wind speed reduces score, max penalty of 30
    score -= wind_penalty

    # Rain scoring
    if rain:
        rain_penalty = 30 if rain > 0.5 else 15  # Heavy rain is a big penalty, light rain less so
        score -= rain_penalty

    # Weather description scoring
    if 'snow' in description.lower() or 'ice' in description.lower():
        score -= 50
    elif 'sun' in description.lower():
        score += 10  # Bonus for sunny weather
    elif 'cloud' in description.lower():
        score -= 5  # Slight penalty for cloudy weather

    # Round the score to 0 decimal places
    score = round(score)

    # Determine conditions summary based on score
    if score >= 90:
        return "Perfect conditions for tennis! 🌞", score
    elif 70 <= score < 90:
        return "Good conditions for tennis!", score
    elif 50 <= score < 70:
        return "Okay conditions; may not be ideal.", score
    else:
        return "Bad conditions for tennis. Consider rescheduling. ⚠️", score


def display_forecast_with_synopsis(forecast):
    temp = forecast['temperature']
    wind_speed = forecast['wind_speed']
    rain = forecast['rain_forecast']
    description = forecast['description']

    # Calculate tennis score and get synopsis
    synopsis, score = calculate_tennis_score(temp, wind_speed, rain, description)

    print(f"Forecast for {forecast['time']}: {description}, "
          f"Temperature: {temp}°C, Wind Speed: {wind_speed} m/s")
    print(synopsis)
    print(f"We give it a tennis weather score of : {score}/100")

def get_multi_day_am_pm_forecast(location):
    # Define the base URL with location as postal code and country code
    base_url = f"http://api.openweathermap.org/data/2.5/forecast?zip={location},GB&appid={API_KEY}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    if data['cod'] != '200':
        print(f"Failed to retrieve data for {location}. Please check the location and try again.")
        return None

    # Organize data by day and time period (AM or PM)
    daily_data = defaultdict(lambda: {"AM": [], "PM": []})
    for entry in data['list']:
        forecast_time = datetime.fromtimestamp(entry['dt'])
        day = forecast_time.date()
        period = "AM" if forecast_time.hour < 12 else "PM"
        daily_data[day][period].append(entry)

    # Process daily summaries for AM and PM
    forecast_summary = {}
    for day, periods in daily_data.items():
        day_summary = {}
        for period, entries in periods.items():
            if entries:
                temp_sum = 0
                max_wind_speed = 0
                rain_forecast = False

                for entry in entries:
                    temp_sum += entry['main']['temp']
                    wind_speed = entry['wind']['speed']
                    max_wind_speed = max(max_wind_speed, wind_speed)
                    if 'rain' in entry and entry['rain'].get('3h', 0) > 0:
                        rain_forecast = True

                avg_temp = temp_sum / len(entries)
                day_summary[period] = {
                    "average_temp": avg_temp,
                    "max_wind_speed": max_wind_speed,
                    "rain_forecast": rain_forecast
                }
        forecast_summary[day] = day_summary

    return forecast_summary

def display_multi_day_forecast(forecast_summary):
    print("Multi-day AM/PM Tennis Weather Forecast:")
    for day, periods in forecast_summary.items():
        print(f"\n{day}:")
        for period, summary in periods.items():
            rain_message = "Rain expected" if summary['rain_forecast'] else "No rain expected"
            synopsis, score = calculate_tennis_score(
                summary['average_temp'], summary['max_wind_speed'], summary['rain_forecast'], "clear sky"
            )
            print(f"  {period}: Avg Temp: {summary['average_temp']:.1f}°C, "
                  f"Max Wind Speed: {summary['max_wind_speed']:.1f} m/s, {rain_message}")
            print(f"  {synopsis} (Score: {score}/100)")

def get_forecast_or_multi_day():
    location = input("Enter the first part of your postal code (e.g., SW6 for Parsons Green): ")
    choice = input("Type '1' for a specific time forecast, '2' for a multi-day AM/PM summary: ")

    if choice == '1':
        play_time = input("Enter the time you plan to play (e.g., '2024-11-05 15:00'): ")
        play_time_dt = datetime.strptime(play_time, '%Y-%m-%d %H:%M')
        forecast = get_forecast(location, play_time_dt)
        if forecast:
            display_forecast_with_synopsis(forecast)
    elif choice == '2':
        forecast_summary = get_multi_day_am_pm_forecast(location)
        if forecast_summary:
            display_multi_day_forecast(forecast_summary)
    else:
        print("Invalid choice. Please try again.")

# Run the app
get_forecast_or_multi_day()


Enter the first part of your postal code (e.g., SW6 for Parsons Green): SW6
Type '1' for a specific time forecast, '2' for a multi-day AM/PM summary: 1
Enter the time you plan to play (e.g., '2024-11-05 15:00'): 2024-11-14 19:00
Forecast for 2024-11-14 18:00:00: clear sky, Temperature: 9.63°C, Wind Speed: 1.33 m/s
Good conditions for tennis!
We give it a tennis weather score of : 76/100


Email version - remove option 1, send out daily email with 5 day AM and PM forecast

yxqy dabb scmw zryb

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests
from datetime import datetime
from collections import defaultdict

# OpenWeather API setup
API_KEY = 'fd94c86864c1809c326f7f0b6add6acc'  # Replace with your OpenWeather API key

def calculate_tennis_score(temp, wind_speed_mph, rain, description):
    score = 100
    ideal_temp = 15
    temp_deviation = abs(temp - ideal_temp)
    score -= min(temp_deviation * 2, 20)

    wind_penalty = min(wind_speed_mph * 2, 30)  # Higher wind speed reduces score, max penalty of 30
    score -= wind_penalty

    if rain:
        rain_penalty = 30 if rain > 0.5 else 15
        score -= rain_penalty

    if 'snow' in description.lower() or 'ice' in description.lower():
        score -= 50
    elif 'sun' in description.lower():
        score += 10
    elif 'cloud' in description.lower():
        score -= 5

    score = round(score)
    reason = ""

    if temp_deviation > 5:
        reason = "Looking chilly out there!" if temp < ideal_temp else "Might be hot!"
    if wind_speed_mph >= 10:
        reason = "Watch out for that wind!"
    if rain:
        reason = "Wet Wet Wet!"
    if 'snow' in description.lower() or 'ice' in description.lower():
        reason = "Don't be Bambi on ice!"

    # If reason is still empty, provide a positive message
    if not reason:
        reason = "It's looking like a great day for a hit!"

    return reason, score

def get_multi_day_am_pm_forecast(postcode):
    base_url = f"http://api.openweathermap.org/data/2.5/forecast?zip={postcode},GB&appid={API_KEY}&units=metric"
    response = requests.get(base_url)
    data = response.json()
    if data['cod'] != '200':
        print(f"Failed to retrieve data for {postcode}. Please check the postcode and try again.")
        return None
    daily_data = defaultdict(lambda: {"AM": [], "PM": []})
    for entry in data['list']:
        forecast_time = datetime.fromtimestamp(entry['dt'])
        day = forecast_time.date()
        period = "AM" if forecast_time.hour < 12 else "PM"
        daily_data[day][period].append(entry)
    forecast_summary = {}
    for day, periods in daily_data.items():
        day_summary = {}
        for period, entries in periods.items():
            if entries:
                temp_sum = 0
                max_wind_speed_mph = 0
                total_rain_mm = 0
                for entry in entries:
                    temp_sum += entry['main']['temp']
                    wind_speed_mph = entry['wind']['speed'] * 2.23694  # Convert m/s to mph
                    max_wind_speed_mph = max(max_wind_speed_mph, wind_speed_mph)
                    total_rain_mm += entry.get('rain', {}).get('3h', 0)  # Get rain in mm
                avg_temp = temp_sum / len(entries)
                day_summary[period] = {
                    "average_temp": avg_temp,
                    "max_wind_speed_mph": max_wind_speed_mph,
                    "total_rain_mm": total_rain_mm
                }
        forecast_summary[day] = day_summary
    return forecast_summary

def determine_best_day(forecast_summary):
    best_day = None
    highest_score = -1
    for day, periods in forecast_summary.items():
        for period, summary in periods.items():
            reason, score = calculate_tennis_score(
                summary['average_temp'], summary['max_wind_speed_mph'], summary['total_rain_mm'], "clear sky"
            )
            if score > highest_score:
                highest_score = score
                best_day = (day, period, reason, score)
    if best_day:
        day, period, reason, score = best_day
        day_name = day.strftime("%A")  # Define day_name using strftime
        return f"The best time to play tennis is on {day_name} ({day}) during the {period} with a score of {score}/100. {reason}"
    else:
        return "No suitable day for playing tennis in the next 5 days."

def create_html_email_content(forecast_summary, postcode):
    with open("/content/drive/MyDrive/Colab Notebooks/tennis_weather/templates/email_template.html", "r") as file:
        template = file.read()

    content_rows = ""
    for day, periods in forecast_summary.items():
        day_name = day.strftime("%A")  # Get the day name from the date
        for period, summary in periods.items():
            rain_message = f"{summary['total_rain_mm']:.1f} mm of rain" if summary['total_rain_mm'] > 0 else "No rain expected"
            reason, score = calculate_tennis_score(
                summary['average_temp'], summary['max_wind_speed_mph'], summary['total_rain_mm'], "clear sky"
            )
            content_rows += f"""
                <tr>
                    <td>{day} ({day_name})</td>
                    <td>{period}</td>
                    <td>{summary['average_temp']:.1f}°C</td>
                    <td>{summary['max_wind_speed_mph']:.1f} mph</td>
                    <td>{rain_message}</td>
                    <td>{reason}</td>
                    <td>{score}/100</td>
                </tr>
            """

    best_day_recommendation = determine_best_day(forecast_summary)
    html_content = template.replace("SW6", postcode)  # Replace SW6 with the actual postcode
    html_content = html_content.replace("{{content_rows}}", content_rows)
    html_content = html_content.replace("{{best_day_recommendation}}", best_day_recommendation)
    return html_content

def send_email(content, postcode):
    sender_email = "jamierjhill@gmail.com"  # Replace with your email
    receiver_email = "jamierjhill@gmail.com"  # Replace with recipient's email
    password = "yxqy dabb scmw zryb"  # Replace with your email password

    msg = MIMEMultipart("alternative")
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f"Your Multi-day Tennis Weather Forecast for {postcode} 🎾"
    msg.attach(MIMEText(content, 'html'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.send_message(msg)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

def get_and_send_forecast():
    postcode = "SW6"  # You can replace this with any postcode you want
    forecast_summary = get_multi_day_am_pm_forecast(postcode)
    if forecast_summary:
        content = create_html_email_content(forecast_summary, postcode)
        send_email(content, postcode)
    else:
        print("Failed to get the forecast. Please try again.")

# Run the app
get_and_send_forecast()


Email sent successfully!


Future state  - create wesbite with user login, ability to input postcode and receive a 5 day forecast.